In [1]:
import numpy as np
from music21 import converter, instrument, note, chord, meter

In [37]:
"""We modeled the monophonic melodies and basslines as
sequences of 16th note events. This resulted in a 130-
dimensional output space (categorical distribution over tokens) with 128 “note-on” tokens for the 128 MIDI pitches,
plus single tokens for “note-off” and “rest”. For drum patterns, we mapped the 61 drum classes defined by the General
MIDI standard (International MIDI Association, 1991) to 9
canonical classes and represented all possible combinations
of hits with 512 categorical tokens. For timing, in all cases
we quantized notes to 16th note intervals, such that each
bar consisted of 16 events. As a result, our two-bar data
(used as a proof-of-concept with a flat decoder) resulted in
sequences with T = 32 and 16-bar data had T = 256. For
our hierarchical models, we use U = 16, meaning each
subsequence corresponded to a single bar"""

midi_path = 'data/16th_allnotes_oneinstrument.mid'

midi = converter.parse(midi_path)
midi.timeSignature = meter.TimeSignature('1/8')
print(midi.timeSignature)
notes = midi.flat.notes

for element in notes:
    if isinstance(element, note.Note):
        print("{}\t{}\t{}".format(element, element.pitch.midi, element.beatDuration))
    else:
        #element.duration.type = 'eighth'
        print("{}\t{}\t{}".format(element, [p.midi for p in element.pitches], [n.beatDuration for n in element.notes]))

<music21.meter.TimeSignature 1/8>


Music21ObjectException: this object does not have a TimeSignature in Sites

In [8]:
from note_seq import midi_file_to_note_sequence, midi_file_to_melody, melody_encoder_decoder
melody = midi_file_to_melody('data/midis/16th_allnotes_oneinstrument.mid')

one_hot = melody_encoder_decoder.MelodyOneHotEncoding(0,124)
one_hot.encode_event(melody)

TypeError: '<' not supported between instances of 'Melody' and 'int'

In [10]:
from magenta.scripts.convert_dir_to_note_sequences import convert_directory

convert_directory('data/midis', 'data/test.tfrecord')

INFO:tensorflow:Converting files in 'data/midis\'.
INFO:tensorflow:0 files converted.
INFO:tensorflow:Converted MIDI file data/midis\16th_allnotes_oneinstrument.mid.
INFO:tensorflow:Converted MIDI file data/midis\8th_allnotes_oneinstrument.mid.
